In [ ]:
import duckdb
import dtale
import pandas as pd

In [ ]:
APP_USAGE_PATH = "data/dataset_app_usage.csv"
WEB_USAGE_PATH = "data/dataset_web_usage.csv"

con = duckdb.connect()
df_app = con.execute(f"SELECT * FROM read_csv_auto('{APP_USAGE_PATH}')").df()
df_web = con.execute(f"SELECT * FROM read_csv_auto('{WEB_USAGE_PATH}')").df()

In [ ]:
print(df_app.shape)
df_app.head()

In [ ]:
df_app.info()
df_app.describe()

In [ ]:
print(df_web.shape)
df_web.head()

In [ ]:
df_web.info()
df_web.describe()

In [ ]:
# Use dtale library to dig deeper into the data
dtale.show(df_app, open_browser=True)
dtale.show(df_web, open_browser=True)

### AUFGABEN

## Aufgabe 1

In [ ]:
# Web usage: Was fällt dir am Leseverhalten am Wochenende (Samstag, Sonntag) auf? Wie würdest du dir das erklären?

con = duckdb.connect()

SQL_QUERY_TASK1 = f"""
WITH web_usage AS (

    SELECT DISTINCT
        to_timestamp("Timestamp") AS event_timestamp,
        "Device" as device_type,
        "User_ID" AS user_id, 
        "Visits" AS visits,
        "Pageviews" AS page_views,
        "Visit_Duration" AS visit_duration
                      
    FROM read_csv_auto('{WEB_USAGE_PATH}')
),

add_temporal_features AS (

SELECT 

    *,

    -- temporal features
    CAST(event_timestamp AS DATE) AS event_date,
    YEAR(event_timestamp) as event_year,
    MONTH(event_timestamp) as event_month,
    DAYNAME(event_timestamp) as event_dayname,
    ISODOW(event_timestamp) as event_day_of_week,
        case 
        when ISODOW(event_timestamp) in (6, 7) then 1
        else 0
    end as event_is_weekend,
    hour(event_timestamp) as event_hour,

FROM web_usage
)

SELECT

    event_day_of_week,
    event_dayname,
    
    count(distinct user_id) AS total_users,
    sum(visits) AS total_visits_sum,
    sum(page_views) AS total_page_views,
    avg(visit_duration) AS avg_visit_duration,
    avg(page_views / visits) AS avg_page_views_per_visit

FROM add_temporal_features

GROUP BY
1, 2
ORDER BY 
1   

"""

df_task1 = con.execute(SQL_QUERY_TASK1).df()

df_task1.to_csv("data/mart/task1_web_usage_weekend.csv", index=False)

In [ ]:
# viszualize total page views by day of the week and average visit duration in one plot
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax1 = plt.subplots(figsize=(10, 6))
sns.barplot(
    x='event_dayname',
    y='total_page_views',
    data=df_task1,
    color='tab:blue',
    ax=ax1
)
ax1.set_ylabel('Total Page Views', color='tab:blue')
ax1.set_xlabel('Day of the Week')
ax1.tick_params(axis='y', labelcolor='tab:blue')

ax2 = ax1.twinx()
sns.lineplot(
    x='event_dayname',
    y='avg_visit_duration',
    data=df_task1,
    color='tab:orange',
    marker='o',
    ax=ax2
)
ax2.set_ylabel('Avg Visit Duration (seconds)', color='tab:orange')
ax2.tick_params(axis='y', labelcolor='tab:orange')

ax3 = ax1.twinx()
sns.lineplot(
    x='event_dayname',
    y='avg_page_views_per_visit',
    data=df_task1,
    color='tab:red',
    marker='o',
    ax=ax3
)
ax3.set_ylabel('', color='tab:red')
ax3.tick_params(axis='y', labelcolor='tab:red')

plt.title('Total Page Views and Avg Visit Duration by Day of the Week')
fig.tight_layout()
plt.show()

## Aufgabe 2

In [ ]:
# Web usage: Zu welcher Tageszeit sind die meisten Nutzer auf der Seite? Kommen die gleichen User, die morgens die Setie besuchen, auch abends nochmal wieder?

con = duckdb.connect()

SQL_QUERY_TASK2 = f"""
WITH web_usage AS (

    SELECT DISTINCT
        to_timestamp("Timestamp") AS event_timestamp,
        "Device" as device_type,
        "User_ID" AS user_id, 
        "Visits" AS visits,
        "Pageviews" AS page_views,
        "Visit_Duration" AS visit_duration
                      
    FROM read_csv_auto('{WEB_USAGE_PATH}')
),

add_temporal_features AS (

SELECT 

    *,

    -- temporal features
    CAST(event_timestamp AS DATE) AS event_date,
    hour(event_timestamp) as event_hour,
    DAYNAME(event_timestamp) as event_dayname,
    ISODOW(event_timestamp) as event_day_of_week,
    case
        when hour(event_timestamp) >= 6 and hour(event_timestamp) < 12 then 'morning'
        when hour(event_timestamp) >= 12 and hour(event_timestamp) < 18 then 'afternoon'
        when hour(event_timestamp) >= 18 and hour(event_timestamp) < 24 then 'evening'
        else 'night'
    end as event_time_of_day,
    case 
        when ISODOW(event_timestamp) in (6, 7) then 1
        else 0
    end as event_is_weekend
   
FROM web_usage
)

SELECT

    event_time_of_day,
    event_hour,
    
    count(distinct user_id) AS total_users,
    sum(visits) AS total_visits_sum,
    sum(page_views) AS total_page_views,
    avg(visit_duration) AS avg_visit_duration,
    avg(page_views / visits) AS avg_page_views_per_visit

FROM add_temporal_features

GROUP BY
1,2
ORDER BY 
2

"""

df_task2 = con.execute(SQL_QUERY_TASK2).df()

df_task2.to_csv("data/mart/task2_web_usage_daytime.csv", index=False)



In [ ]:
# viszualize total page views by hour
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax1 = plt.subplots(figsize=(10, 6))
sns.barplot(
    x='event_hour',
    y='total_users',
    data=df_task2,
    color='tab:blue',
    ax=ax1
)
ax1.set_ylabel('Total Users', color='tab:blue')
ax1.set_xlabel('Hour of the Day')
ax1.tick_params(axis='y', labelcolor='tab:blue')

plt.title('Total Users by Hour of the Day')
fig.tight_layout()
plt.show()